In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
from tqdm.auto import tqdm
import functools

/root/miniconda3/envs/auto_pantip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [132]:
driver.get('https://dictionary.cambridge.org/dictionary/english/although')

In [7]:
element = driver.find_element('xpath', '/html/body/div[2]/div/div[1]/div[2]/article/div[2]/div[1]/div[2]/div/div[3]/div/div')

In [8]:
element

<selenium.webdriver.remote.webelement.WebElement (session="07a0c43fed984d2643e98de972e284b5", element="aaf8cde7-d94b-4184-bb50-bed0aae80257")>

In [9]:
soup = BeautifulSoup(str(driver.page_source), 'html.parser')
p = soup.find_all('div', class_='entry-body')

In [134]:
p[0]

<div class="entry-body"> <div class="pr entry-body__el"><div class="cid" id="cald4-1"></div><div class="pos-header dpos-h"><div class="di-title"><span class="headword hdb tw-bw dhw dpos-h_hw"><span class="hw dhw">perfect</span></span></div><div class="posgram dpos-g hdib lmr-5"><span class="pos dpos" title="A word that describes a noun or pronoun.">adjective</span></div> <div></div><span class="uk dpron-i"><span class="region dreg">uk</span><span class="daud">
<audio class="hdn" controlslist="nodownload" id="audio1" preload="none">
<div class="hdib" fallback="">
<p>Your browser doesn't support HTML5 audio</p>
</div>
<source src="/media/english/uk_pron/u/ukp/ukper/ukperch016.mp3" type="audio/mpeg"/>
<source src="/media/english/uk_pron_ogg/u/ukp/ukper/ukperch016.ogg" type="audio/ogg"/>
</audio>
<div class="i i-volume-up c_aud htc hdib hp hv-1 fon tcu tc-bd lmr-10 lpt-3 fs20 hv-3" onclick="audio1.load(); audio1.play();" role="button" tabindex="0" title="Listen to the British English pronu

In [43]:
list(p[0].find_all('div', class_='pr entry-body__el')[2].find('div', class_='def ddef_d db').children)

['to make something ',
 <a class="query" href="https://dictionary.cambridge.org/dictionary/english/free" title="free">free</a>,
 ' from ',
 <a class="query" href="https://dictionary.cambridge.org/dictionary/english/fault" title="faults">faults</a>,
 ': ']

In [107]:
list(p[0].find_all('div', class_='pr entry-body__el')[0].find_all('div', class_='def-block')[0].find('div', class_='def ddef_d db').children)[-1].text

': '

['complete',
 ' and ',
 'correct',
 ' in every way, of the ',
 'best',
 ' ',
 'possible',
 ' ',
 'type',
 ' or without ',
 'fault',
 ': ']

In [82]:
p[0].find_all('span', class_='pos dpos')

[<span class="pos dpos" title="A word that describes a noun or pronoun.">adjective</span>,
 <span class="pos dpos" title="A word that refers to a person, place, idea, event or thing.">noun</span>,
 <span class="pos dpos" title="A word that describes an action, condition or experience.">verb</span>]

In [126]:
oxford_3000 = open('oxford_3000.txt', 'r').read().split('\n')
oxford_5000 = open('oxford_5000.txt', 'r').read().split('\n')

In [115]:
# oxford_3000 = ['perfect']

In [131]:
vocab_list_3000 = []
oxford3000_loop = tqdm(oxford_3000)

for _ in oxford3000_loop :
    oxford3000_loop.set_description(f'Word: {_}')
    driver.get(f'https://dictionary.cambridge.org/dictionary/english/{_}')
    soup = BeautifulSoup(str(driver.page_source), 'html.parser')
    main_result = soup.find_all('div', class_='entry-body')[0]
    sub_result = main_result.find_all('div', class_='pr entry-body__el')
    for i in sub_result :
        pos = i.find('span', class_='pos dpos').text
        sound = i.find('span', class_='ipa').text
        try : 
            cefr_level = i.find_all('div', class_='def-block')[0].find('span', class_='epp-xref').text
        except :
            cefr_level = 'none'
        definition = functools.reduce(lambda x,y: x+y, [ _x.text for _x in i.find_all('div', class_='def-block')[0].find('div', class_='def ddef_d db').children ]).replace(': ', '') + '.'
        vocab_list_3000.append({
            'vocab' : _,
            'sound' : sound,
            'pos' : pos,
            'cefr_level' : cefr_level,
            'definition' : definition,
        })
        

Word: although:   3%|▎         | 94/2978 [02:36<1:19:56,  1.66s/it]      


IndexError: list index out of range

In [135]:
pd.DataFrame(vocab_list_3000)

,vocab,sound,pos,cefr_level,definition
0,a,eɪ,noun,none,the first letter of the English alphabet.
1,abandon,əˈbæn.dən,verb,B2,"to leave a place, thing, or person, usually fo..."
2,abandon,əˈbæn.dən,noun,none,in a completely uncontrolled way.
3,ability,əˈbɪl.ə.ti,noun,B1,the physical or mental power or skill needed t...
4,able,ˈeɪ.bəl,adjective,A2,"to have the necessary physical strength, menta..."
...,...,...,...,...,...
120,already,ɔːlˈred.i,adverb,A2,before the present time.
121,also,ˈɔːl.səʊ,adverb,A1,in addition.
122,alter,ˈɒl.tər,verb,B2,"to change something, usually slightly, or to c..."
123,alternative,ɒlˈtɜː.nə.tɪv,adjective,B2,An alternative plan or method is one that you ...


In [27]:
word_df = pd.read_csv('vocab_5000.csv')

In [28]:
word_df

,vocab,sound,pos,cefr_level,definition
0,a,eɪ,noun,none,the first letter of the English alphabet.
1,abandon,əˈbæn.dən,verb,B2,"to leave a place, thing, or person, usually fo..."
2,abandon,əˈbæn.dən,noun,none,in a completely uncontrolled way.
3,ability,əˈbɪl.ə.ti,noun,B1,the physical or mental power or skill needed t...
4,able,ˈeɪ.bəl,adjective,A2,"to have the necessary physical strength, menta..."
...,...,...,...,...,...
6167,yell,jel,verb,B2,"to shout something or make a loud noise, usual..."
6168,yell,jel,noun,B2,"a loud shout, usually when you are angry, in p..."
6169,yield,jiːld,verb,C2,to supply or produce something positive such a...
6170,yield,jiːld,noun,none,"an amount of something positive, such as food ..."


In [8]:
word_df[word_df['cefr_level'] == 'C1']

,vocab,sound,pos,cefr_level,definition
29,acknowledge,əkˈnɒl.ɪdʒ,verb,C1,"to accept, admit, or recognize something, or t..."
51,administration,ədˌmɪn.ɪˈstreɪ.ʃən,noun,C1,the arrangements and tasks needed to control t...
86,agenda,əˈdʒen.də,noun,C1,a list of matters to be discussed at a meeting.
95,aid,eɪd,verb,C1,to help.
104,alarm,əˈlɑːm,verb,C1,to make someone worried or frightened.
...,...,...,...,...,...
6152,withdraw,wɪðˈdrɔː,verb,C1,"to take or move out or back, or to remove."
6156,workplace,ˈwɜːk.pleɪs,noun,C1,a building or room where people perform their ...
6161,worship,ˈwɜː.ʃɪp,verb,C1,to have or show a strong feeling of respect an...
6164,worthy,ˈwɜː.ði,adjective,C1,"deserving respect, admiration, or support."


In [9]:
c1_df = word_df[word_df['cefr_level'] == 'C1']

In [12]:
c1_df['word'] = c1_df.apply(lambda _: _['vocab'] + ' (' + _['pos'] + ')', axis=1)
c1_df['answer'] = c1_df.apply(lambda _: _['sound'] + '\n' + _['definition'], axis=1)

/tmp/ipykernel_684369/1207442910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c1_df['word'] = c1_df.apply(lambda _: _['vocab'] + ' (' + _['pos'] + ')', axis=1)
/tmp/ipykernel_684369/1207442910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c1_df['answer'] = c1_df.apply(lambda _: _['sound'] + '\n' + _['definition'], axis=1)


In [15]:
c1_df[['word', 'answer']].to_csv('c1_df.csv', index=False)

In [3]:
import json

In [4]:
unsuccess_list = json.loads(open('unsuccess.txt', 'r').read())

In [6]:
unsuccess_list.__len__()

713

In [17]:
# unsuccess_list_vocab = []
unsuccess_list_4 = []
oxford3000_loop = tqdm(unsuccess_list_3)

for _ in oxford3000_loop :
    try :
        oxford3000_loop.set_description(f'Word: {_}')
        driver.get(f'https://dictionary.cambridge.org/dictionary/english/{_}')
        soup = BeautifulSoup(str(driver.page_source), 'html.parser')
        main_result = soup.find_all('div', class_='entry-body')[0]
        sub_result = main_result.find_all('div', class_='pr entry-body__el')
        for i in sub_result :
            pos = i.find('span', class_='pos dpos').text
            sound = i.find('span', class_='ipa').text
            try : 
                cefr_level = i.find_all('div', class_='def-block')[0].find('span', class_='epp-xref').text
            except :
                cefr_level = 'none'
            definition = functools.reduce(lambda x,y: x+y, [ _x.text for _x in i.find_all('div', class_='def-block')[0].find('div', class_='def ddef_d db').children ]).replace(': ', '') + '.'
            unsuccess_list_vocab.append({
                'vocab' : _,
                'sound' : sound,
                'pos' : pos,
                'cefr_level' : cefr_level,
                'definition' : definition,
            })

    except :
        unsuccess_list_4.append(_)
        

Word: proceeding: 100%|██████████| 10/10 [01:00<00:00,  6.08s/it]


In [18]:
unsuccess_list_4.__len__()

10

In [19]:
unsuccess_list_4

['impressed',
 'insurance',
 'located',
 'pattern',
 'platform',
 'power',
 'sublist',
 'addiction',
 'cute',
 'proceeding']

In [21]:
unsuccess_df = pd.DataFrame(unsuccess_list_vocab)

In [25]:
unsuccess_df.drop_duplicates()

,vocab,sound,pos,cefr_level,definition
0,budget,ˈbʌdʒ.ɪt,noun,B2,a plan to show how much money a person or orga...
1,budget,ˈbʌdʒ.ɪt,verb,none,to plan how much money you will spend on somet...
2,budget,ˈbʌdʒ.ɪt,adjective,B2,very cheap.
3,build,bɪld,verb,A2,to make something by putting bricks or other m...
4,build,bɪld,noun,none,the size and shape of a person's body.
...,...,...,...,...,...
1025,object,ˈɒb.dʒɪkt,noun,B1,a thing that you can see or touch but that is ...
1026,object,əbˈdʒekt,verb,B2,to feel or express opposition to or dislike of...
1027,objective,əbˈdʒek.tɪv,noun,B2,something that you plan to do or achieve.
1028,objective,əbˈdʒek.tɪv,adjective,B2,based on real facts and not influenced by pers...


In [29]:
word_df = pd.concat([word_df, unsuccess_df])

In [32]:
word_df = word_df.drop_duplicates()

In [35]:
word_df.to_csv('vocab_5000.csv',  index=False)

In [36]:
word_df

,vocab,sound,pos,cefr_level,definition
0,a,eɪ,noun,none,the first letter of the English alphabet.
1,abandon,əˈbæn.dən,verb,B2,"to leave a place, thing, or person, usually fo..."
2,abandon,əˈbæn.dən,noun,none,in a completely uncontrolled way.
3,ability,əˈbɪl.ə.ti,noun,B1,the physical or mental power or skill needed t...
4,able,ˈeɪ.bəl,adjective,A2,"to have the necessary physical strength, menta..."
...,...,...,...,...,...
1025,object,ˈɒb.dʒɪkt,noun,B1,a thing that you can see or touch but that is ...
1026,object,əbˈdʒekt,verb,B2,to feel or express opposition to or dislike of...
1027,objective,əbˈdʒek.tɪv,noun,B2,something that you plan to do or achieve.
1028,objective,əbˈdʒek.tɪv,adjective,B2,based on real facts and not influenced by pers...


In [44]:
word_df['vocab'] = word_df['vocab'].apply(lambda x: x.lower())

/tmp/ipykernel_771869/2813914810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df['vocab'] = word_df['vocab'].apply(lambda x: x.lower())


In [46]:
word_df['word'] = word_df.apply(lambda _: _['vocab'] + ' (' + _['pos'] + ')', axis=1)
word_df['answer'] = word_df.apply(lambda _: _['sound'] + '\n' + _['definition'], axis=1)

/tmp/ipykernel_771869/1091876135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df['word'] = word_df.apply(lambda _: _['vocab'] + ' (' + _['pos'] + ')', axis=1)
/tmp/ipykernel_771869/1091876135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_df['answer'] = word_df.apply(lambda _: _['sound'] + '\n' + _['definition'], axis=1)


In [50]:
word_df[['vocab', 'sound', 'pos', 'cefr_level', 'definition']].to_csv('oxford_5000_processed.csv')

In [51]:
word_df = word_df.sort_values(by='vocab', ascending=True)

In [54]:
word_df[['word', 'answer']].to_csv('output/oxford_5000_all.csv', index=False)

In [64]:
word_df[word_df['cefr_level'] == 'C2'][['word', 'answer']].to_csv('output/oxford_cefr_C2.csv', index=False)